In [11]:
import dimod
import json

In [12]:
def load_qubo(file_path):
    with open(file_path, "r") as f:
        all_qubos_data = json.load(f)  
    return all_qubos_data

def convert_to_dimod_format(all_qubos_data):
    dimod_qubos = {}
    for instance in all_qubos_data:
        instance_id = instance["instance_id"]
        terms = instance["terms"]
        weights = instance["weights"]

        # Create the QUBO dictionary for the current instance
        Q = {}
        for term, weight in zip(terms, weights):
            if len(term) == 1:
                # This is a linear term (e.g., [0]), format as ('x0', 'x0')
                variable = f'x{term[0]}'
                Q[(variable, variable)] = weight
            elif len(term) == 2:
                # This is a quadratic term (e.g., [0, 1]), format as ('x0', 'x1')
                var1 = f'x{term[0]}'
                var2 = f'x{term[1]}'
                Q[(var1, var2)] = weight

        dimod_qubos[instance_id] = Q
    return dimod_qubos

def convert_to_dimod_ising(all_qubos_data):
    dimod_ising = {}
    for instance in all_qubos_data:
        instance_id = instance["instance_id"]
        terms = instance["terms"]
        weights = instance["weights"]

        linear_bias = {}
        quadratic_bias = {}

        for term, weight in zip(terms, weights):
            if len(term) == 1:
                variable = f'x{term[0]}'
                linear_bias[variable] = weight
            elif len(term) == 2:
                var1 = f'x{term[0]}'
                var2 = f'x{term[1]}'
                quadratic_bias[(var1, var2)] = weight

        dimod_ising[instance_id] = (linear_bias, quadratic_bias)
    return dimod_ising

In [13]:
#////////// variables //////////
#quboBatchFile = "QUBO_batches/batch_QUBO_data_Knapsack_6_items_9q_.json"
#quboBatchFile = "QUBO_batches/batch_QUBO_data_TSP_9q_.json"
quboBatchFile = 'qubo_data.json'
targetInstance = 1

In [17]:
qubos = load_qubo(quboBatchFile)
dimodQubos = convert_to_dimod_format(qubos)
dimodIsings = convert_to_dimod_ising(qubos)
print(dimodIsings)
#print(f'linearBias: {linearBias}, quadraticBias: {quadraticBias}')

{1: ({}, {('x0', 'x4'): 1.0, ('x1', 'x2'): 1.0, ('x1', 'x3'): 1.0, ('x1', 'x4'): 1.0, ('x2', 'x3'): 1.0, ('x2', 'x4'): 1.0, ('x3', 'x4'): 1.0})}


In [ ]:
print(dimodQubos[targetInstance])

{('x0', 'x4'): 1.0, ('x1', 'x2'): 1.0, ('x1', 'x3'): 1.0, ('x1', 'x4'): 1.0, ('x2', 'x3'): 1.0, ('x2', 'x4'): 1.0, ('x3', 'x4'): 1.0}


In [19]:
Q = dimodQubos[targetInstance]
I = dimodIsings[targetInstance]
print(I)

# Use the exact solver
solver = dimod.ExactSolver()
sampleset = solver.sample_ising(I[0], I[1])

# The first sample in the result is the optimal solution
print("--- D-Wave Ocean Exact Solver ---")
print(sampleset)

({}, {('x0', 'x4'): 1.0, ('x1', 'x2'): 1.0, ('x1', 'x3'): 1.0, ('x1', 'x4'): 1.0, ('x2', 'x3'): 1.0, ('x2', 'x4'): 1.0, ('x3', 'x4'): 1.0})
--- D-Wave Ocean Exact Solver ---
   x0 x1 x2 x3 x4 energy num_oc.
4  -1 +1 -1 -1 +1   -3.0       1
9  +1 +1 +1 -1 -1   -3.0       1
12 -1 -1 +1 -1 +1   -3.0       1
17 +1 -1 +1 +1 -1   -3.0       1
25 +1 +1 -1 +1 -1   -3.0       1
28 -1 -1 -1 +1 +1   -3.0       1
3  -1 -1 -1 -1 +1   -1.0       1
5  +1 +1 -1 -1 +1   -1.0       1
6  +1 +1 -1 -1 -1   -1.0       1
8  -1 +1 +1 -1 -1   -1.0       1
11 -1 +1 +1 -1 +1   -1.0       1
13 +1 -1 +1 -1 +1   -1.0       1
14 +1 -1 +1 -1 -1   -1.0       1
16 -1 -1 +1 +1 -1   -1.0       1
19 -1 -1 +1 +1 +1   -1.0       1
22 +1 +1 +1 +1 -1   -1.0       1
24 -1 +1 -1 +1 -1   -1.0       1
27 -1 +1 -1 +1 +1   -1.0       1
29 +1 -1 -1 +1 +1   -1.0       1
30 +1 -1 -1 +1 -1   -1.0       1
2  +1 -1 -1 -1 +1    1.0       1
7  -1 +1 -1 -1 -1    1.0       1
10 +1 +1 +1 -1 +1    1.0       1
15 -1 -1 +1 -1 -1    1.0       1
1